## Exploratory Data Analysis (EDA) for Netflix Subscriber Churn Prediction

In [14]:
#Important needed libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [3]:
#Load dataset 
netflix_data = pd.read_csv("../data/netflix_raw.csv")

### Initial Data Inspection

The goal of this step is to understand the structure, scale, and basic attributes of the dataset before performing any cleaning or analysis.

In [11]:
#Print first few rows of dataset then see # of rows & columns in entire dataset.
print(netflix_data.head())
print("-------------------------------------------------------------------------------------------")
print("Number of rows, columns: ") 
print(netflix_data.shape)

                            customer_id  age  gender subscription_type  \
0  a9b75100-82a8-427a-a208-72f24052884a   51   Other             Basic   
1  49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b   47   Other          Standard   
2  4d71f6ce-fca9-4ff7-8afa-197ac24de14b   27  Female          Standard   
3  d3c72c38-631b-4f9e-8a0e-de103cad1a7d   53   Other           Premium   
4  4e265c34-103a-4dbb-9553-76c9aa47e946   56   Other          Standard   

   watch_hours  last_login_days   region  device  monthly_fee  churned  \
0        14.73               29   Africa      TV         8.99        1   
1         0.70               19   Europe  Mobile        13.99        1   
2        16.32               10     Asia      TV        13.99        0   
3         4.51               12  Oceania      TV        17.99        1   
4         1.89               13   Africa  Mobile        13.99        1   

  payment_method  number_of_profiles  avg_watch_time_per_day favorite_genre  
0      Gift Card                

In [12]:
# Generate summary statistics on numerical variables 
print(netflix_data.describe())
print("-------------------------------------------------------------------------------------------")
print(netflix_data.info())

               age  watch_hours  last_login_days  monthly_fee      churned  \
count  5000.000000  5000.000000      5000.000000  5000.000000  5000.000000   
mean     43.847400    11.649450        30.089800    13.683400     0.503000   
std      15.501128    12.014654        17.536078     3.692062     0.500041   
min      18.000000     0.010000         0.000000     8.990000     0.000000   
25%      30.000000     3.337500        15.000000     8.990000     0.000000   
50%      44.000000     8.000000        30.000000    13.990000     1.000000   
75%      58.000000    16.030000        45.000000    17.990000     1.000000   
max      70.000000   110.400000        60.000000    17.990000     1.000000   

       number_of_profiles  avg_watch_time_per_day  
count         5000.000000             5000.000000  
mean             3.024400                0.874800  
std              1.415841                2.619824  
min              1.000000                0.000000  
25%              2.000000            

## Observations 

- The dataset contains ~5,000 records with an even split of numeric and categorical features
- No missing values are present in the core numeric fields.
- The target variable (`churned`) is binary, suggesting a classification problem.
- Several categorical variables (e.g., subscription_type, device, region) will require encoding.

### Next Steps

Based on the initial exploration, the next phase will focus on:
- Handling categorical variables
- Feature engineering
- Preparing the dataset for model training